In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Cardio_Data_CleanUp").getOrCreate()

### Load Data into Spark DataFrame

In [106]:
import os
os.getcwd()

'/content'

In [28]:
from pyspark import SparkFiles
csv_file_path = "/content/Resources/Cardio_Data.csv"
cardio_df_original = spark.read.csv(csv_file_path, header=True, mode="DROPMALFORMED")
cardio_df_original.show()
cardio_df_original.printSchema()

+-----+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|   id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+-----+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|82567|18804|     2|   165|    10|  180| 1100|          2|   2|    0|   0|     1|     1|
|48318|21582|     2|   178|    11|  130|   90|          1|   1|    0|   0|     1|     1|
|85931|21855|     1|   162|    21|  120|   80|          2|   1|    0|   0|     1|     1|
|42156|20408|     2|   177|    22|  120|   80|          1|   1|    1|   1|     1|     0|
|38312|23284|     1|   157|    23|  110|   80|          1|   1|    0|   0|     1|     0|
|48976|14664|     2|   128|    28|  120|   80|          1|   1|    0|   0|     1|     0|
|86650|18875|     1|   171|    29|  110|   70|          2|   1|    0|   0|     1|     1|
| 5306|15400|     1|   120|    30|  110|   70|          1|   1|    0|   0|     1|     0|
|26503|18140|     1| 

### Analysis of DataFrame

In [5]:
# Description of columns
cardio_df_original.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------+-------------------+------------------+
|summary|                id|               age|            gender|            height|            weight|             ap_hi|             ap_lo|       cholesterol|              gluc|              smoke|                alco|             active|            cardio|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+--------------------+-------------------+------------------+
|  count|             70000|             70000|             70000|             70000|             70000|             70000|             70000|             70000|             70000|              70000|               70

In [29]:
# Change the column names
cardio_df = cardio_df_original.withColumnRenamed("ap_hi", "systolic_bp").withColumnRenamed("ap_lo", "diastolic_bp")


In [30]:
cardio_df.show()

+-----+-----+------+------+------+-----------+------------+-----------+----+-----+----+------+------+
|   id|  age|gender|height|weight|systolic_bp|diastolic_bp|cholesterol|gluc|smoke|alco|active|cardio|
+-----+-----+------+------+------+-----------+------------+-----------+----+-----+----+------+------+
|82567|18804|     2|   165|    10|        180|        1100|          2|   2|    0|   0|     1|     1|
|48318|21582|     2|   178|    11|        130|          90|          1|   1|    0|   0|     1|     1|
|85931|21855|     1|   162|    21|        120|          80|          2|   1|    0|   0|     1|     1|
|42156|20408|     2|   177|    22|        120|          80|          1|   1|    1|   1|     1|     0|
|38312|23284|     1|   157|    23|        110|          80|          1|   1|    0|   0|     1|     0|
|48976|14664|     2|   128|    28|        120|          80|          1|   1|    0|   0|     1|     0|
|86650|18875|     1|   171|    29|        110|          70|          2|   1|    0|

In [31]:
# Check for null rows
Dict_Null = {col:cardio_df.filter(cardio_df[col].isNull()).count() for col in cardio_df.columns}
Dict_Null


{'active': 0,
 'age': 0,
 'alco': 0,
 'cardio': 0,
 'cholesterol': 0,
 'diastolic_bp': 0,
 'gender': 0,
 'gluc': 0,
 'height': 0,
 'id': 0,
 'smoke': 0,
 'systolic_bp': 0,
 'weight': 0}

In [32]:
# Check for duplicate rows 
cardio_df.distinct().count()

70000

In [97]:
# Check the distinct values of the categorical variables
# https://stackoverflow.com/questions/64805788/get-distinct-values-of-multiple-columns
from pyspark.sql.functions import collect_set
from pyspark.sql.functions import col

In [99]:
columns = ["gender", "cholesterol", "gluc", "smoke", "alco", "active", "cardio"]
columnExprs = map(lambda c: collect_set(col(c)).alias(c), columns)
cardio_df.select(*columnExprs).show()

+------+-----------+---------+------+------+------+------+
|gender|cholesterol|     gluc| smoke|  alco|active|cardio|
+------+-----------+---------+------+------+------+------+
|[1, 2]|  [3, 1, 2]|[3, 1, 2]|[1, 0]|[1, 0]|[1, 0]|[1, 0]|
+------+-----------+---------+------+------+------+------+



### Data Processing on continous variables

In [39]:
# Filter based on height. Keep height between 135 - 215cm
cardio_df = cardio_df.filter((cardio_df["height"] >=135) & (cardio_df["height"] <=215))

In [42]:
# The shape of the dataframe after filtering out the improbable values of height
print((cardio_df.count(), len(cardio_df.columns)))

(69883, 13)


In [43]:
# Filter based on weight. Keep height between 25 - 200kg
cardio_df = cardio_df.filter((cardio_df["weight"] >=25) & (cardio_df["weight"] <=200))

In [68]:
# The shape of the dataframe after filtering out the improbable values of weight
print((cardio_df.count(), len(cardio_df.columns)))

(69878, 13)


In [66]:
# Change the signs of negative values of systolic and diastolic bp
from pyspark.sql.functions import abs
cardio_df = cardio_df.withColumn("systolic_bp", abs(cardio_df["systolic_bp"])).withColumn("diastolic_bp", abs(cardio_df["diastolic_bp"]))

In [67]:
# Check for presence of negative values in the columns
cardio_df.filter((cardio_df["diastolic_bp"] <0)).groupby("diastolic_bp").count().show()
cardio_df.filter((cardio_df["systolic_bp"] <0)).groupby("systolic_bp").count().show()


+------------+-----+
|diastolic_bp|count|
+------------+-----+
+------------+-----+

+-----------+-----+
|systolic_bp|count|
+-----------+-----+
+-----------+-----+



In [87]:
# Filter based on systolic_bp values between 80 and 180. 
cardio_df = cardio_df.filter((cardio_df["systolic_bp"] >=80) & (cardio_df["systolic_bp"] <=180))

In [88]:
# Filter based on diastolic_bp values between 40 and 120. 
cardio_df = cardio_df.filter((cardio_df["diastolic_bp"] >=40) & (cardio_df["diastolic_bp"] <=120))

In [89]:
# The shape of the dataframe after filtering out the improbable values of systolic and diastolic bp
print((cardio_df.count(), len(cardio_df.columns)))

(68297, 13)


### Storing data into output csv file

In [105]:
# Write to output csvfile.
# First convert to a pandas_df
cardio_cleaned_df = cardio_df.toPandas()

# Store result
cardio_cleaned_df.to_csv("/content/Resources/cardio_data_cleaned.csv")